In [28]:
#Detects emotions from text
#Created by S. Biswas
import pandas as pd
import numpy as np

In [29]:
df_train = pd.read_csv('train.txt', sep=';', names=['Sentence', 'Emotion'], encoding='UTF8')
#data cleaning
df_train = df_train[df_train['Sentence'] != '']
df_train = df_train.dropna()
df_train = df_train.drop_duplicates()
# df_train = df_train[df_train['Sentence'] != '']


In [30]:
df_train['length'] = [len(sen) for sen in df_train['Sentence']]

In [31]:
# df_train.head()

In [32]:
# import matplotlib.pyplot as plt
# import seaborn as sea
# sea.countplot(x=df_train['Emotion'])
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df_train['Emotion'] = lb.fit_transform(df_train['Emotion'])

In [33]:
# print(df_train['Emotion'])

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
df_train['Cleaned_Sentence'] = df_train['Sentence'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
df_train['Cleaned_Sentence'] = df_train['Cleaned_Sentence'].apply(lambda x: x.lower())
df_train['Final_Cleaned_Sentence'] = df_train['Cleaned_Sentence'].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split() if word not in stopwords))



In [35]:
df_train.head()

,Sentence,Emotion,length,Cleaned_Sentence,Final_Cleaned_Sentence
0,i didnt feel Humiliated,4,23,i didnt feel humiliated,didnt feel humili
1,i can go from feeling so hopeless to so damned...,4,108,i can go from feeling so hopeless to so damned...,go feel hopeless damn hope around someon care ...
2,im grabbing a minute to post i feel greedy wrong,0,48,im grabbing a minute to post i feel greedy wrong,im grab minut post feel greedi wrong
3,i am ever feeling nostalgic about the fireplac...,3,92,i am ever feeling nostalgic about the fireplac...,ever feel nostalg fireplac know still properti
4,i am feeling grouchy,0,20,i am feeling grouchy,feel grouchi


In [36]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['Final_Cleaned_Sentence'], df_train['Emotion'], test_size= 0.2 , random_state=42 )


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer()
x_train_tfidf = tfidfv.fit_transform(x_train)
x_test_tfidf = tfidfv.transform(x_test)
print(x_train_tfidf)

  (0, 8853)	0.31232674165251334
  (0, 6835)	0.31232674165251334
  (0, 7972)	0.20517199765788086
  (0, 6475)	0.1764283970540595
  (0, 4991)	0.31232674165251334
  (0, 4404)	0.31232674165251334
  (0, 7465)	0.22465437747659178
  (0, 4784)	0.31232674165251334
  (0, 6695)	0.2901547609783056
  (0, 4198)	0.28301697747997845
  (0, 7456)	0.20517199765788086
  (0, 2901)	0.032412553141535384
  (0, 3684)	0.15613857137659465
  (0, 1147)	0.1604098943217147
  (0, 8145)	0.16139419987817885
  (0, 1746)	0.21731088647469238
  (0, 6555)	0.2403137555932144
  (1, 8288)	0.4671569474614646
  (1, 8039)	0.43951216075960775
  (1, 7459)	0.4153149781387904
  (1, 550)	0.5174465558324732
  (1, 9117)	0.3485419971324422
  (1, 2901)	0.16393429784069966
  (2, 4690)	0.40099387094537087
  (2, 7709)	0.35087417987605735
  :	:
  (12794, 6493)	0.4274935959265569
  (12794, 1620)	0.39207774264510764
  (12794, 4598)	0.34162795753036035
  (12794, 7327)	0.3778581392591926
  (12794, 4788)	0.2531631000602922
  (12794, 2901)	0.0627621

In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

classifier = {
    'MultinomialNB' : MultinomialNB(),
    'SVC':SVC(),
    'RandomForestClassifier' : RandomForestClassifier(),
    'LogisticRegression' : LogisticRegression(),   
}
for name, cls in classifier.items():
    cls.fit(x_train_tfidf, y_train)
    y_pred = cls.predict(x_test_tfidf)
    acc = accuracy_score(y_test, y_pred)
    print(f"====={name}======\n")
    print(f"====={acc}======\n")
    rep = classification_report(y_test, y_pred)
    print(f"======{rep}=====")
    
    

=====MultinomialNB======

=====0.6590625======

======              precision    recall  f1-score   support

           0       0.95      0.32      0.48       439
           1       0.88      0.23      0.37       375
           2       0.58      0.98      0.73      1027
           3       1.00      0.03      0.05       303
           4       0.72      0.91      0.80       950
           5       1.00      0.02      0.04       106

    accuracy                           0.66      3200
   macro avg       0.85      0.42      0.41      3200
weighted avg       0.76      0.66      0.59      3200
=====
=====SVC======

=====0.815625======

======              precision    recall  f1-score   support

           0       0.88      0.79      0.83       439
           1       0.84      0.73      0.78       375
           2       0.75      0.94      0.83      1027
           3       0.82      0.37      0.51       303
           4       0.87      0.90      0.89       950
           5       0.82      0

/home/sauman/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:


# Assuming stemmer and tfidfv are already defined and trained
lg = LogisticRegression()
lg.fit(x_train_tfidf, y_train)

def clean_data_for_predict(text):
    clean_text = re.sub("[^a-zA-Z]", " ", text)
    t = clean_text.lower()  # Apply lowercasing on clean_text

    # Ensure stemmer is called correctly
    t = ' '.join(stemmer.stem(w) for w in t.split())
    return t

def predict_emotion(text):
    text = clean_data_for_predict(text)
    v = tfidfv.transform([text])  # Wrap text in a list to form a 2D array
    label = lg.predict(v)[0]  # Get the predicted label directly
    emo = lb.inverse_transform(lg.predict(v))[0]
    return emo, label



/home/sauman/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
predict_emotion("I make you")

('joy', 2)